# ISE Software Testing: Lab 5

## Property Testing

In this lab, we're going to use a property-based testing library to test our code. By now, you're used to writing tests that check specific inputs and outputs. Property-based testing is a different approach to testing that allows you to specify properties that your code should satisfy, and then the testing library will generate random inputs and check that your code satisfies the properties for all of them.

### Setup

First, let's install the `hypothesis` library (https://hypothesis.readthedocs.io/en/latest/). This is a property-based testing library for Python. It's not part of the standard library, so we need to install it separately.

In [36]:
%pip install hypothesis

import math
import unittest
import typing
from hypothesis import given, strategies as st, Verbosity, settings

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### A Simple Example

Let's start with a simple example! If we're checking that the `sin` function is working correctly, we might quickly run into a problem. Only a few inputs are easy to check by hand, while the rest are difficult to calculate. We can't just check that `sin(0) == 0` and `sin(1) == 1` and call it a day. Instead, we can exploit the fact that `sin` is periodic, and check that `sin(x) == sin(x + 2*pi)` for all `x`. This is a property that we expect `sin` to satisfy.

In [37]:
def run_test_case(test_case: typing.Type[unittest.TestCase]):
    """This is a helper function to run a test case in a Jupyter notebook."""
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(test_case)
    unittest.TextTestRunner().run(suite)


class SineTest(unittest.TestCase):
    @given(st.floats(min_value=0, max_value=2 * math.pi))
    def test_sine(self, x):
        # The sine function is periodic with period 2*pi
        # Let's exploit this property to test the sine function
        self.assertAlmostEqual(math.sin(x), math.sin(x + 2 * math.pi), places=10)


run_test_case(SineTest)

.
----------------------------------------------------------------------
Ran 1 test in 0.054s

OK


### A Useful Example

Talk is cheap, let's try a more useful example. Remember `find_last` from the first lab? Finding a failing test case for that function was awkward, because we had to try a bunch of different inputs before we found one that failed. Let's try to use property-based testing to find a failing test case for `find_last`.

This time around, we don't need to generate any inputs ourselves. We just need to specify the properties that we want our code to satisfy. In this case, we want to check that `find_last` satisfies some properties (commented in the source code below). In the code below, we also set the verbosity to `Verbosity.verbose` so that we can see the inputs that are being generated and tested.

Run the code below. You should see a bunch of inputs being generated and tested. Eventually, you should see a failing test case. What is the failing test case? What is the expected output? What is the actual output? What is the difference between the expected and actual output?

Fix the bug in `find_last` and re-run the code. You should see that all of the tests pass now.

In [39]:
def find_last(arr, y):
    """Find the index of the last occurrence of y in the given list."""
    for i in range(len(arr) - 1, 0, -1):
        if arr[i] == y:
            return i
    return -1


class FindLastTest(unittest.TestCase):
    @given(st.lists(st.integers()), st.integers())
    @settings(verbosity=Verbosity.verbose)
    def test_find_last(self, arr, y):
        # Find the last occurrence of y in arr
        last_index = find_last(arr, y)
        if last_index != -1:
            # Check that y is actually at that index in the array
            self.assertEqual(arr[last_index], y)
            # Check that there is no other occurrence of y after the last_index
            self.assertNotIn(y, arr[last_index + 1 :])
        else:
            # Check that there is no occurrence of y in arr
            self.assertNotIn(y, arr)


run_test_case(FindLastTest)

.

Trying example: test_find_last(
    self=<__main__.FindLastTest testMethod=test_find_last>,
    arr=[0],
    y=0,
)
Trying example: test_find_last(
    self=<__main__.FindLastTest testMethod=test_find_last>,
    arr=[0],
    y=0,
)
Trying example: test_find_last(
    self=<__main__.FindLastTest testMethod=test_find_last>,
    arr=[],
    y=0,
)
Trying example: test_find_last(
    self=<__main__.FindLastTest testMethod=test_find_last>,
    arr=[0],
    y=0,
)
Trying example: test_find_last(
    self=<__main__.FindLastTest testMethod=test_find_last>,
    arr=[0],
    y=0,
)
Trying example: test_find_last(
    self=<__main__.FindLastTest testMethod=test_find_last>,
    arr=[14933,
     10334,
     -177,
     4550344568873485490,
     1115,
     -22623,
     1698378323,
     -32602,
     6578],
    y=-102,
)
Trying example: test_find_last(
    self=<__main__.FindLastTest testMethod=test_find_last>,
    arr=[-3313],
    y=0,
)
Trying example: test_find_last(
    self=<__main__.FindLastTest 


----------------------------------------------------------------------
Ran 1 test in 0.092s

OK
